In [181]:
import pandas as pd

airports = pd.read_csv('data/airports.csv')
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [238]:
airportLocations = {}
for index, row in airports.iterrows():
    airportLocations[row["IATA_CODE"]] = (row["LATITUDE"], row["LONGITUDE"])
airportLocations

{'ABE': (40.652359999999994, -75.4404),
 'ABI': (32.41132, -99.6819),
 'ABQ': (35.04022, -106.60918999999998),
 'ABR': (45.449059999999996, -98.42183),
 'ABY': (31.53552, -84.19447),
 'ACK': (41.25305, -70.06018),
 'ACT': (31.611290000000004, -97.23052),
 'ACV': (40.978120000000004, -124.10862),
 'ACY': (39.45758, -74.57717),
 'ADK': (51.877959999999995, -176.64603),
 'ADQ': (57.74997, -152.49386),
 'AEX': (31.327370000000002, -92.54856),
 'AGS': (33.36996, -81.9645),
 'AKN': (58.6768, -156.64921999999999),
 'ALB': (42.74812, -73.80298),
 'ALO': (42.55708, -92.40034),
 'AMA': (35.21937, -101.70593000000001),
 'ANC': (61.174319999999994, -149.99618999999998),
 'APN': (45.078070000000004, -83.56029000000001),
 'ASE': (39.22316, -106.86885),
 'ATL': (33.640440000000005, -84.42694),
 'ATW': (44.25741, -88.51948),
 'AUS': (30.19453, -97.66986999999999),
 'AVL': (35.43619, -82.54181),
 'AVP': (41.33815, -75.72426999999999),
 'AZO': (42.23488, -85.55206),
 'BDL': (41.93887, -72.68323000000001

In [239]:
import cPickle
import datetime

origin = 'HNL'
destination = 'BOS'
direct = False

if direct:
    policyFile = origin + 'to' + destination + '-direct_policy.pkl'
    valueFile = origin + 'to' + destination + '-direct_value.pkl'
else:
    policyFile = origin + 'to' + destination + '_policy.pkl'
    valueFile = origin + 'to' + destination + '_value.pkl'

print 'loading optimal policy pkl'
optimal_policy = {}
with open(policyFile, "rb") as input_file:
    optimal_policy = cPickle.load(input_file)
print 'done loading optimal policy'

print 'loading value function pkl'
value_function = {}
with open(valueFile, "rb") as input_file:
    value_function = cPickle.load(input_file)
print 'done loading value function'

state = (origin, datetime.datetime(2015, 1, 11, 8, 30), 0)
path = [(state, None)]

while True:
    print '\n'
    print 's: ', state, value_function[state]
    if state[0] == destination:
        break
    else:
        action = optimal_policy[state]
        print 'a: ', action
        state = (action[2],action[3],state[2]+1)
        path.append((state,action,state[2]+1))

airportsVisited = [i[0][0] for i in path]
airportsVisited

loading optimal policy pkl


IOError: [Errno 2] No such file or directory: 'HNLtoBOS_policy.pkl'

In [240]:
totalValue = [value_function[i[0]] for i in path]
totalValue

[-955.4922908111605, -376.24316166993322, 0]

In [241]:
flightsToTake = [(i[1][0],i[1][4]) for i in path[1:]]
flightsToTake

[('UA1231', 337.0), ('VX360', 307.0)]

In [242]:
points = []
for airport in airportsVisited:
    points.append(airportLocations[airport])
points

[(21.31869, -157.92241),
 (33.94254, -118.40807),
 (42.36435, -71.00518000000001)]

In [243]:
import matplotlib.pyplot as plt
import mplleaflet

lats = [i[0] for i in points]
lngs = [i[1] for i in points]

if direct:
    plt.plot(lngs, lats,'ob-')
else:
    plt.plot(lngs, lats,'or-')

mplleaflet.show()